In [1]:
from utils import *
from scipy.spatial.distance import cdist

# Handling NA / NaN

In [2]:
for key, df in dfs.items():
    nas = df.isna().sum().replace(0,np.nan).dropna() / df.shape[0] * 100
    print(key, '\n\t', nas)

transfer 
	 Series([], dtype: float64)
statement 
	 Series([], dtype: float64)
transaction_statement 
	 Series([], dtype: float64)
transaction 
	 trans_spending_category    11.411191
transaction_type            1.958687
merch_x_coord              55.284444
merch_y_coord              55.284444
dtype: float64
customer_demog 
	 customer_income_level    8.0
dtype: float64
branch_visit 
	 Series([], dtype: float64)


**Impute customer demographic nans** only as we don't use the other missing categories in our final dataset

In [3]:
median_val = np.nanmedian(dfs.get('customer_demog')['customer_income_level'])
dfs.get('customer_demog')['customer_income_level'] = dfs.get('customer_demog')['customer_income_level'].replace(np.nan, median_val)

In [4]:
for key, df in dfs.items():
    nas = df.isna().sum().replace(0,np.nan).dropna() / df.shape[0] * 100
    print(key, '\n\t', nas)

transfer 
	 Series([], dtype: float64)
statement 
	 Series([], dtype: float64)
transaction_statement 
	 Series([], dtype: float64)
transaction 
	 trans_spending_category    11.411191
transaction_type            1.958687
merch_x_coord              55.284444
merch_y_coord              55.284444
dtype: float64
customer_demog 
	 Series([], dtype: float64)
branch_visit 
	 Series([], dtype: float64)


# Feature Engineering

## Transfers

In [5]:
df_transfer = dfs.get('transfer').copy()

## Branch Visits

In [393]:
df_branch = dfs.get('branch_visit').copy()

Converts a visit date and time to categorical (dummy encoded) day of week and categorical (dummy encoded) time of day

In [394]:
df_visit_time = pd.to_datetime(df_branch['branch_visit_time'], format='%d%b%Y:%H:%M:%S.000000')
day_of_week = pd.get_dummies(df_visit_time.dt.dayofweek, prefix = 'BV_DoW') #Branch Visit Day of Week
time_of_day = pd.get_dummies(pd.cut(df_visit_time.dt.hour, bins=4)) #Branch Visit Time of Day
time_of_day.columns = ['BV_very_early','BV_early','BV_later','BV_late']

df_branch = df_branch.join(day_of_week).join(time_of_day)
df_branch = df_branch.drop(['branch_visit_date','branch_visit_time','branch_id'],axis=1)
df_branch = df_branch.groupby('customer_id')[df_branch.columns[2:]].sum()
df_branch.head()

,branch_x_coord,branch_y_coord,BV_DoW_0,BV_DoW_1,BV_DoW_2,BV_DoW_3,BV_DoW_4,BV_DoW_5,BV_DoW_6,BV_very_early,BV_early,BV_later,BV_late
customer_id,,,,,,,,,,,,,
2853662,519.952292,398.227655,6,2,1,2,1,1,0,8,4,0,1
3542321,402.041058,302.288704,2,2,3,1,2,0,0,3,6,0,1
4365610,779.085500,554.020905,4,5,4,2,4,0,0,13,6,0,0
9021512,40.994090,28.794200,0,1,0,0,0,0,0,0,0,1,0
9339300,155.839902,128.967988,1,2,0,0,1,0,0,0,3,1,0


Finding centered branch location and the average distance from the center (distance not currently working)

In [395]:
df_branch['branch_x_mean'] = df_branch.groupby('customer_id')['branch_x_coord'].transform('mean')
df_branch['branch_y_mean'] = df_branch.groupby('customer_id')['branch_y_coord'].transform('mean')
branch_mean = df_branch.groupby('customer_id')[['branch_y_mean','branch_x_mean','branch_x_coord','branch_y_coord']].apply(
    lambda df: cdist(df[['branch_x_coord','branch_y_coord']], np.array([df[['branch_x_mean','branch_y_mean']].iloc[0,:]])).mean()).to_frame()
branch_mean.columns = ['branch_mean_distance']

df_branch = df_branch.merge(branch_mean,on='customer_id',how='left')
df_branch.head()

,branch_x_coord,branch_y_coord,BV_DoW_0,BV_DoW_1,BV_DoW_2,BV_DoW_3,BV_DoW_4,BV_DoW_5,BV_DoW_6,BV_very_early,BV_early,BV_later,BV_late,branch_x_mean,branch_y_mean,branch_mean_distance
customer_id,,,,,,,,,,,,,,,,
2853662,519.952292,398.227655,6,2,1,2,1,1,0,8,4,0,1,519.952292,398.227655,0.0
3542321,402.041058,302.288704,2,2,3,1,2,0,0,3,6,0,1,402.041058,302.288704,0.0
4365610,779.085500,554.020905,4,5,4,2,4,0,0,13,6,0,0,779.085500,554.020905,0.0
9021512,40.994090,28.794200,0,1,0,0,0,0,0,0,0,1,0,40.994090,28.794200,0.0
9339300,155.839902,128.967988,1,2,0,0,1,0,0,0,3,1,0,155.839902,128.967988,0.0


In [396]:
df_branch = df_branch.drop(['branch_x_coord','branch_y_coord'],axis=1)
df_branch.head()

,BV_DoW_0,BV_DoW_1,BV_DoW_2,BV_DoW_3,BV_DoW_4,BV_DoW_5,BV_DoW_6,BV_very_early,BV_early,BV_later,BV_late,branch_x_mean,branch_y_mean,branch_mean_distance
customer_id,,,,,,,,,,,,,,
2853662,6,2,1,2,1,1,0,8,4,0,1,519.952292,398.227655,0.0
3542321,2,2,3,1,2,0,0,3,6,0,1,402.041058,302.288704,0.0
4365610,4,5,4,2,4,0,0,13,6,0,0,779.085500,554.020905,0.0
9021512,0,1,0,0,0,0,0,0,0,1,0,40.994090,28.794200,0.0
9339300,1,2,0,0,1,0,0,0,3,1,0,155.839902,128.967988,0.0


## Payments

In [386]:
df_payment = dfs.get('transaction_statement').copy()
df_payment.set_index('customer_id', inplace = True)
df_payment.head()

,cc_id,payment_date,statement_currency,payment_amount
customer_id,,,,
15438869,142251658,31DEC2014:00:00:00,TURK_LIRASI,1857.71
9021512,29807547,16JUN2015:00:00:00,TURK_LIRASI,4290.00
16792108,148247128,31DEC2014:00:00:00,TURK_LIRASI,650.00
15717336,128192194,17JUN2015:00:00:00,TURK_LIRASI,280.00
19858904,140913499,17JUN2015:00:00:00,TURK_LIRASI,4386.00


In [387]:
df_payment=df_payment[df_payment.statement_currency=='TURK_LIRASI']

# Get the date range of transaction data; used to calculate monthly averages
dates = pd.to_datetime(df_payment['payment_date'], format='%d%b%Y:%H:%M:%S')
transaction_range = (dates.max().to_period('M') - dates.min().to_period('M')).n + 1

In [389]:
# Average amount paid for any given payment
payment_mean = df_payment.groupby('customer_id')['payment_amount'].mean()

# Variation of payment amount
payment_std = df_payment.groupby('customer_id')['payment_amount'].std()

# Average times customer makes a payment in a month
payment_freq = df_payment.groupby('customer_id')['payment_amount'].count() / transaction_range

In [392]:
df_payment = pd.concat([payment_mean, payment_std, payment_freq], axis = 1, keys = ['payment_mean', 'payment_std', 'payment_monthly_freq'])
df_payment.head()

,payment_mean,payment_std,payment_monthly_freq
customer_id,,,
2782253,1579.741667,729.641388,1.000000
2853662,1620.760000,600.893342,0.833333
3054783,934.529091,772.183352,1.833333
3542321,5804.265000,2019.900721,1.000000
3908957,969.028333,750.814448,1.000000


## Transactions

In [368]:
df_transactions = dfs.get('transaction').copy()
df_transactions.head()

,customer_id,transaction_date,transaction_time,transaction_total,trans_spending_category,merch_id,online_payment,transaction_type,currency,merch_x_coord,merch_y_coord
0,4837694,27OCT2014:00:00:00,10:49:38,50.00,SEYAHAT ACENTALARI - TAŞIMACILIK,999999,0,AV,TURK_LIRASI,NaN,NaN
1,4837694,13DEC2014:00:00:00,11:05:31,700.00,NaN,999999,0,AV,TURK_LIRASI,NaN,NaN
2,4837694,15NOV2014:00:00:00,14:14:15,500.00,TELEKOMÜNİKASYON,999999,0,AV,TURK_LIRASI,NaN,NaN
3,4837694,20OCT2014:00:00:00,16:29:26,1600.00,TELEKOMÜNİKASYON,999999,0,AV,TURK_LIRASI,NaN,NaN
4,4837694,30OCT2014:00:00:00,20:10:31,19.45,GIDA,999999,0,AV,TURK_LIRASI,NaN,NaN


Grabbing the top 9 spending categories

In [369]:
tran_cat_list = df_transactions.trans_spending_category.value_counts(sort = True)[:9].index
tran_cat_list

Index(['GIDA', 'RESTORAN', 'TEKSTİL', 'AKARYAKIT', 'SAĞLIK', 'DİĞER',
       'HİZMET SEKTÖRLERİ', 'TELEKOMÜNİKASYON', 'TEKNOLOJİ'],
      dtype='object')

In [370]:
df_transactions['trans_spending_category'] = df_transactions['trans_spending_category'].apply(lambda i: i if i in tran_cat_list else 'OTHER')
df_transactions.trans_spending_category.value_counts()

GIDA                 4207
OTHER                3899
RESTORAN             2484
TEKSTİL              1870
AKARYAKIT             753
SAĞLIK                538
DİĞER                 350
HİZMET SEKTÖRLERİ     345
TELEKOMÜNİKASYON      297
TEKNOLOJİ             216
Name: trans_spending_category, dtype: int64

The sample dataset was largely transactions in one currency. The others we consider outliers (since so small).

In [371]:
df_transactions=df_transactions[df_transactions.currency=='TURK_LIRASI']

Prep for calculation

In [372]:
# Get the date range of transaction data; used to calculate monthly averages
dates = pd.to_datetime(df_transactions['transaction_date'], format='%d%b%Y:%H:%M:%S')
transaction_range = (dates.max().to_period('M') - dates.min().to_period('M')).n + 1

# Grab relevant columns
df_transactions = df_transactions.get(['customer_id', 'trans_spending_category','transaction_total'])

df_transactions.set_index(['customer_id','trans_spending_category'], inplace = True)
df_transactions.head()

transaction_total
customer_id trans_spending_category                   
4837694     OTHER                                50.00
            OTHER                               700.00
            TELEKOMÜNİKASYON                    500.00
            TELEKOMÜNİKASYON                   1600.00
            GIDA                                 19.45

Average amount in one given transaction and the monthly frequency of transactions

In [373]:
transaction_average_amount = df_transactions.groupby(['customer_id','trans_spending_category'])['transaction_total'].mean()
transaction_average_monthly_frequency = df_transactions.groupby(['customer_id','trans_spending_category'])['transaction_total'].count()/transaction_range
df_transactions = pd.concat([transaction_average_amount, transaction_average_monthly_frequency], axis = 1, keys = ["trans_average_amount","trans_average_monthly_freq"])
df_transactions.head()

trans_average_amount  \
customer_id trans_spending_category                         
2782253     AKARYAKIT                          216.010000   
            GIDA                                46.102500   
            OTHER                              817.205000   
            RESTORAN                            80.000000   
            TEKNOLOJİ                           84.993333   

                                     trans_average_monthly_freq  
customer_id trans_spending_category                              
2782253     AKARYAKIT                                  0.083333  
            GIDA                                       2.666667  
            OTHER                                      1.666667  
            RESTORAN                                   0.083333  
            TEKNOLOJİ                                  0.250000

In [374]:
df_transactions = df_transactions.reset_index().pivot(index = 'customer_id', columns = 'trans_spending_category')
df_transactions.columns = df_transactions.columns.map('_'.join)
df_transactions.head()

,trans_average_amount_AKARYAKIT,trans_average_amount_DİĞER,trans_average_amount_GIDA,trans_average_amount_HİZMET SEKTÖRLERİ,trans_average_amount_OTHER,trans_average_amount_RESTORAN,trans_average_amount_SAĞLIK,trans_average_amount_TEKNOLOJİ,trans_average_amount_TEKSTİL,trans_average_amount_TELEKOMÜNİKASYON,trans_average_monthly_freq_AKARYAKIT,trans_average_monthly_freq_DİĞER,trans_average_monthly_freq_GIDA,trans_average_monthly_freq_HİZMET SEKTÖRLERİ,trans_average_monthly_freq_OTHER,trans_average_monthly_freq_RESTORAN,trans_average_monthly_freq_SAĞLIK,trans_average_monthly_freq_TEKNOLOJİ,trans_average_monthly_freq_TEKSTİL,trans_average_monthly_freq_TELEKOMÜNİKASYON
customer_id,,,,,,,,,,,,,,,,,,,,
2782253,216.010000,NaN,46.102500,NaN,817.205000,80.000000,NaN,84.993333,56.103846,45.0,0.083333,NaN,2.666667,NaN,1.666667,0.083333,NaN,0.250000,1.083333,0.416667
2853662,139.948636,241.103333,33.876154,NaN,216.409600,NaN,174.312500,1449.000000,74.965000,NaN,1.833333,0.25,1.083333,NaN,2.083333,NaN,0.333333,0.166667,0.333333,NaN
3054783,NaN,NaN,35.970000,165.9125,152.542769,29.535000,21.500000,13.990000,102.613437,NaN,NaN,NaN,4.166667,0.333333,5.416667,1.666667,0.083333,0.083333,5.333333,NaN
3542321,87.916667,486.385556,20.322529,240.9000,285.546963,24.943958,17.341667,855.772857,73.438636,30.0,5.000000,1.50,29.000000,0.333333,11.250000,12.000000,0.500000,0.583333,1.833333,0.083333
3908957,75.000000,NaN,58.525714,61.6000,296.164211,47.260000,NaN,29.900000,505.130000,NaN,0.333333,NaN,1.166667,0.166667,3.166667,2.083333,NaN,0.166667,0.416667,NaN


## Statements

In [12]:
df_statement = dfs.get('statement')

df_statement = df_statement.groupby('customer_id')['statement_amount_TL'].mean().to_frame().merge(
    df_statement.groupby('customer_id')['statement_amount_TL'].std().to_frame()
                                                                        , on='customer_id', how='left')
df_statement.columns = ['statement_amount_TL_mean','statement_amount_TL_std']
df_statement

,statement_amount_TL_mean,statement_amount_TL_std
customer_id,,
2782253,1602.015833,699.007006
2853662,804.960000,936.941104
3054783,849.675833,776.237243
3542321,6065.184167,1958.598484
3908957,1226.182500,983.827087
...,...,...
21021028,426.466154,231.552277
21213035,399.039231,378.614187
21242221,212.109310,319.151709


## Customer Demographics

## ATM